In [1]:
import sys
sys.path.append("./")
sys.path.append("./hw2/stud/")
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import config
from tqdm.auto import tqdm
import numpy as np
from utils import *
from load import *
from model import ESC
# import pytorch_lightning as pl


/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
print(config.DEVICE)

cpu


In [4]:
seed_everything(config.SEED)

In [6]:

train_dataset = CoarseGrainedDataset(config.COARSE_TRAIN_DATA)
val_dataset = CoarseGrainedDataset(config.COARSE_VAL_DATA)
test_dataset = CoarseGrainedDataset(config.COARSE_TEST_DATA)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

mapping = load_map(config.MAP_PATH)


/home/francesco/Desktop/University/MNLP/nlp2023-hw2/hw2/stud
/home/francesco/Desktop/University/MNLP/nlp2023-hw2/hw2/stud
/home/francesco/Desktop/University/MNLP/nlp2023-hw2/hw2/stud


In [ ]:
# model
model = ESC()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Loss
criterion = torch.nn.CrossEntropyLoss()

